In [41]:
import requests
from urllib.request import urlopen
from urllib.request import urlretrieve
from bs4 import BeautifulSoup
from datetime import datetime
from PIL import Image
import time
import random
import sqlite3
from config import *

In [53]:
def check_if_dates_are_equal() -> bool:
    dt_format = '%Y-%m-%d'
    conn = sqlite3.connect(db_directory)
    cursor = conn.cursor()
    created_at = cursor.execute(f'SELECT created_at FROM rbc_news WHERE id=0').fetchone()[0]
    conn.commit()
    today_s_date = datetime.now().strftime(format=dt_format)
    print(created_at, today_s_date)
    conn.close()
    return created_at == today_s_date

In [54]:
check_if_dates_are_equal()

2024-10-14 2024-10-14


True

In [42]:
def save_single_article(url:str) -> dict:
    resp = urlopen(url)
    parser = 'lxml'
    bs = BeautifulSoup(resp.read(), parser)    
    article_dict = {}
    article_title_class = 'article__header__title-in js-slide-title'
    article_overview_class = 'article__text__overview'
    article_image_class = 'article__main-image__image'

    article_title = bs.find_all('h1', {'class': article_title_class})[0].contents[0].split('\n')[1].strip()
    article_overview = bs.find_all('div', {'class': article_overview_class})[0].contents[1].contents[0].get_text().replace('«\u200e', '')
    article_image = bs.find_all('img', {'class': article_image_class})[0]['src']

    dt_format = '%Y-%m-%d'
    today = datetime.now().strftime(format=dt_format)

    article_dict['title'] = article_title
    article_dict['overview'] = article_overview
    article_dict['link'] = url
    article_dict['created_at'] = today
    article_dict['image_url'] = article_image

    return article_dict

In [43]:
def save_the_image(img_list:list):
    img_dir = 'images/'
    img_name = img_list[0]
    img_url = img_list[1]
    urlretrieve(img_url, f"{img_dir}{img_name}.jpeg")

In [44]:
def rbc_news_parser():
    txt_file_directory = './database/article_links.txt'
    url = 'https://www.rbc.ru/quote'    
    articles_list = []
    article_links = []
    all_attrs = []
    article_ids_list =  []
    resp = urlopen(url)
    parser = 'lxml'
    bs = BeautifulSoup(resp.read(), parser)    
    all_hrefs = list(bs.find_all('a'))
    for ref in all_hrefs:
        if ref.has_attr('href'):
            all_attrs.append(ref.attrs['href'])
    for a in all_attrs:
        if ('quote/news/article' in a) and ('?from=newsfeed' not in a):
            article_links.append(a)
    article_links = set(article_links)
    # article_links = list(article_links)[:3]

    with open(txt_file_directory, 'r') as f:
        lines = f.readlines()
        set_from_file = set(lines[0].strip(',').split(','))
        # set_from_file = list(set_from_file)[:3]
    print(article_links)
    print(set_from_file)
    print(article_links == set_from_file)
        
    if(article_links != set_from_file):
        with open(txt_file_directory, 'w') as f:
            for el in article_links:
                f.writelines(el + ',')

        for lnk in article_links:
            random_no = random.randint(29, 61)
            single_article = save_single_article(lnk)
            articles_list.append(single_article)
            print(single_article, random_no)
            time.sleep(random_no)
            # print('It is done!')
        # print(set_from_file)

        conn = sqlite3.connect(db_directory)
        cursor = conn.cursor()
        cursor.execute("DELETE FROM rbc_news")
        conn.commit()
        
        for i, article in enumerate(articles_list):
            cursor.execute(f"INSERT INTO rbc_news (id, title, overview, link, created_at, image) VALUES ({i}, '{article['title']}', '{article['overview']}', '{article['link']}', '{article['created_at']}', '{article['image_url']}')")
            print(i)
        conn.commit()
        conn.close()

    conn = sqlite3.connect(db_directory)
    cursor = conn.cursor()

    article_ids = cursor.execute(f'SELECT id, image FROM rbc_news').fetchall()
    
    for art_id in article_ids:
        article_ids_list.append([art_id[0], art_id[1]])
    conn.commit()
    conn.close()
    
    for item in article_ids_list:
        save_the_image(item)
        random_no = random.randint(29, 61)
        time.sleep(random_no)
        print(item, random_no)
    print('It is done!')



In [45]:


rbc_news_parser()


{'https://www.rbc.ru/quote/news/article/6708f5499a79474749d9ab91', 'https://www.rbc.ru/quote/news/article/6708dba99a79471372349c78', 'https://www.rbc.ru/quote/news/article/65b213f69a79477264ef7caf', 'https://www.rbc.ru/quote/news/article/670501e99a7947345c04dbfc', 'https://www.rbc.ru/quote/news/article/67094deb9a79479345ff47fb', 'https://www.rbc.ru/quote/news/article/6700001b9a794773e9ff88f3', 'https://www.rbc.ru/quote/news/article/6709122d9a7947b5fed3e8c6', 'https://www.rbc.ru/quote/news/article/6707b4f29a79470d1fa33322', 'https://www.rbc.ru/quote/news/article/6708e69c9a794716fef1e6be', 'https://www.rbc.ru/quote/news/article/6708b35e9a7947438eec790c', 'https://www.rbc.ru/quote/news/article/6706cde69a7947312477206f', 'https://www.rbc.ru/quote/news/article/670938189a794737dfcd2b7e', 'https://www.rbc.ru/quote/news/article/67077d859a79470f30570dd8', 'https://www.rbc.ru/quote/news/article/6708ffe19a79474a63fdd91b', 'https://www.rbc.ru/quote/news/article/6703d4379a79472bd287829a', 'https://